# Analyze results for 3D images
Jan 21, 2021

In [2]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_3d_image_analysis import *

# sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
# import post_analysis_pandas as post

In [5]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [6]:
img_size=64
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}


### Read validation data and compute metrics

In [7]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'64':raw_data_dir+'3d_data/dataset1_smoothing_const_params_64cube_100k/val.npy'}

In [8]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
s_val=f_transform(s_val)
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

# del(s_val)

(1000, 64, 64, 64)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/modules_3d_image_analysis.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


### Read run data

In [9]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [10]:
u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)

In [21]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202105*')]
q=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(q)

interactive(children=(Dropdown(description='x', options=('20210514_120637_128cube_bs4_lr0.002_nodes16', '20210…

In [22]:
result=q.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210506_92216_bs8_lr0.0003_nodes8_spec0.1


## Plot Losses

In [23]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [24]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
21051,21051.0,134.0,0.392173,0.463315,0.855488,3.577627,14.489210,10.911584,-1.547293,10.950392,-1.206169,NaN,NaN,2.504798,-1.471832,-3.545760,0.208842
21052,21052.0,134.0,0.426469,0.415092,0.841561,1.235093,11.593454,10.358361,-1.381248,10.774064,-0.956787,NaN,NaN,1.263387,-2.685323,-0.853714,0.208759
21053,21053.0,134.0,0.427695,0.632566,1.060261,2.721867,13.495867,10.773999,-1.172922,10.599577,-1.070996,NaN,NaN,2.640104,-0.105372,-2.649389,0.209973
21054,21054.0,134.0,0.383557,0.446604,0.830162,1.197615,11.541973,10.344357,-1.580754,10.530872,-1.368195,NaN,NaN,1.046397,-2.389427,-0.817460,0.208691
21055,21055.0,134.0,0.403037,0.463847,0.866884,3.532295,14.916555,11.384260,-1.873949,11.291126,-1.956727,NaN,NaN,2.494103,-0.658238,-3.499863,0.206993
21056,21056.0,134.0,0.448195,0.440866,0.889060,1.526025,12.896635,11.370609,-1.602237,11.376079,-1.895436,NaN,NaN,1.036837,-2.873784,-1.259249,0.208422
21057,21057.0,134.0,0.425756,0.524776,0.950532,3.325173,14.003904,10.678732,-1.293671,11.233655,-1.385343,NaN,NaN,2.635645,-0.830044,-3.286644,0.214181
21058,21058.0,134.0,0.451699,0.450225,0.901924,1.607469,12.669422,11.061954,-1.503497,11.283251,-1.662388,NaN,NaN,0.805075,-2.960491,-1.370015,0.208462
21059,21059.0,134.0,0.359622,0.559434,0.919056,3.287628,13.893504,10.605876,-1.281682,10.716316,-2.032259,NaN,NaN,2.169192,-0.761454,-3.248551,0.204700
21060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.494166,-1.885492,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [26]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-1.990709297657013


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
20814,20814.0,133.0,0.440388,0.401256,0.841644,1.083512,11.731374,10.647862,-2.115469,10.566220,-2.518323,NaN,NaN,1.750575,-2.350973,-0.622400,0.207991
20575,20575.0,131.0,0.395868,0.376800,0.772669,3.570091,14.555083,10.984993,-1.676899,10.938375,-2.419048,NaN,NaN,1.835557,-1.205345,-3.533757,0.212819
20813,20813.0,133.0,0.393287,0.405956,0.799243,3.246345,14.049175,10.802830,-2.318930,10.751705,-2.412697,NaN,NaN,2.275981,-1.767435,-3.138597,0.207782
19572,19572.0,125.0,0.473126,0.413405,0.886531,3.327310,13.992674,10.665363,-1.120661,10.735760,-2.403017,NaN,NaN,2.961409,-0.947863,-3.286329,0.206977
19030,19030.0,121.0,0.609569,0.567854,1.177423,1.852086,12.648317,10.796231,-2.349277,11.839869,-2.354335,NaN,NaN,0.493840,-2.718169,-1.666462,0.205992
18656,18656.0,119.0,0.318534,0.336202,0.654736,1.481389,12.170753,10.689363,-1.750355,10.781075,-2.344869,NaN,NaN,1.192469,-2.656995,-1.151569,0.205496
19755,19755.0,126.0,0.341524,0.509377,0.850902,3.458720,14.083629,10.624908,-1.972245,10.837195,-2.305877,NaN,NaN,2.441470,-1.387473,-3.423302,0.209147
19001,19001.0,121.0,0.361669,0.483198,0.844867,1.629732,12.221136,10.591405,-1.771638,10.926136,-2.305200,NaN,NaN,0.782746,-2.862103,-1.395563,0.209633
18716,18716.0,119.0,0.390205,0.298025,0.688230,3.619739,14.602551,10.982812,-1.962246,10.891878,-2.298898,NaN,NaN,2.372199,-1.174500,-3.576846,0.207152
20863,20863.0,133.0,0.633277,0.454571,1.087848,1.558105,11.888062,10.329957,-2.236961,10.552377,-2.260524,NaN,NaN,0.768130,-2.892265,-1.298185,0.209149


In [18]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

## Read stored chi-squares for images

In [20]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


FileNotFoundError: [Errno 2] No such file or directory: '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210518_91712_bs8_lr0.001_nodes8_spec0.1/df_processed.pkle'

In [20]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [21]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [22]:
df_sliced=w.result
# df_sliced

In [23]:
best_step=[]
df_test=df.copy()
# df_test=df[df.epoch>5]
best_step.append(f_slice_df(df_test,cutoff=0.9,sort_col='chi_1',head=4,display_flag=False).step.values)
best_step.append(f_slice_df(df_test,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df_test,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 1.5533178654225526, 'chi_spec1': 1927.5022688640038}
[0, 49]
{'chi_1': 0.205462890221009, 'chi_spec1': 962.8806836944607}
[0, 49]
{'chi_1': 0.146272865654264, 'chi_spec1': 121.25042451673932}
[array([ 260, 1440, 1470, 1230]), array([ 200, 1270]), array([1500, 1350])]
[ 200  260 1230 1270 1350 1440 1470 1500]


In [24]:
# best_step=[58800]
# best_step=np.arange(32800,32900,10).astype(int)

In [25]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(8, 22)
[(6, 200), (8, 260), (38, 1230), (39, 1270), (42, 1350), (44, 1440), (45, 1470), (46, 1500)]


In [26]:
df_best[col_list]


,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
19,6-200,0.076222,799.697230,52.590881,0.071650,0.004572,0.0,0.000822,6.088001,35014.828713,6,200
25,8-260,0.022973,1088.839546,52.137237,0.007385,0.015588,0.0,0.000014,8.360224,74869.688577,8,260
122,38-1230,0.045722,31.665181,49.472693,0.041150,0.004572,0.0,0.000207,3.369333,4689.943519,38,1230
126,39-1270,0.192943,897.523654,49.812498,0.188372,0.004572,0.0,0.001794,0.320710,12908.525676,39,1270
134,42-1350,0.081230,24.852898,56.907352,0.076658,0.004572,0.0,0.000668,2.909651,1795.582733,42,1350
143,44-1440,0.035618,27.437536,52.232308,0.031046,0.004572,0.0,0.000191,3.677551,2279.391551,44,1440
146,45-1470,0.040991,24.989060,56.491963,0.036419,0.004572,0.0,0.000729,4.760301,2419.345881,45,1470
149,46-1500,0.090328,23.734970,56.513812,0.085756,0.004572,0.0,0.000450,2.137875,1253.947388,46,1500


### Plots 2: stored data

In [27]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    img_size=64
    assert plot_type in ['hist','spec','grid','spec_relative','hist_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative','hist_relative']:     fig=plt.figure(figsize=(6,6))


    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)


        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'])

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)            
            
            
        if plot_type=='hist_relative':

            x2=row.hist_bin_centers
            y2=row.hist_val
            yerr2=row.hist_err
            x2=f_invtransform(x2)
            
#             plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['hist_val'],dict_bkg['hist_err']

            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'])

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)            
            plt.xlim(0,img_size/2)
            plt.ylim(0,5)
            plt.legend()

        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:,0]
            print(images.shape)
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.xlim(0,img_size/2)
        plt.title('Spectrum')
        
    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        plt.xlim(0,img_size/2)
        plt.ylim(0.5,2)
        plt.legend()
        
    if plot_type=='hist_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [28]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative','hist_relative']))

interactive(children=(SelectMultiple(description='labels_list', options=('6-200', '8-260', '38-1230', '39-1270…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [94]:
print(s_val.shape)


(1000, 64, 64, 64)


In [95]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210123_050147_3d_l0.5_80k/images/gen_img_epoch-28_step-71620.npy'
a1=np.load(fname)

In [96]:
s_val.shape,a1.shape

((1000, 64, 64, 64), (32, 64, 64, 64))

In [97]:
f_plot_grid(a1[6:14,:,:,0],cols=4,fig_size=(4,2))

2 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [98]:
f_plot_grid(s_val[100:108,:,:,0],cols=4,fig_size=(4,2))

2 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [99]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
655,2-6560,0.246885,10.027608,49.063214,0.192645,0.053953,0.000288,0.115025,3.904391,17956.069610,2,6560
2819,11-28200,0.036677,6.014820,49.570766,0.019308,0.014379,0.002990,0.010212,9.884761,1447.230356,11,28200
2920,11-29210,0.248065,6.946046,59.081433,0.241302,0.006456,0.000306,0.089805,33.648975,50.338523,11,29210
3378,13-33790,0.032535,6.317830,50.120442,0.028484,0.002187,0.001864,0.013548,6.696044,1461.858703,13,33790
4015,16-40160,0.032709,5.377288,50.452657,0.027667,0.003498,0.001544,0.013563,10.932140,240.525028,16,40160
4024,16-40250,0.100561,3.085800,43.490086,0.071879,0.025700,0.002983,0.051874,7.510610,71.322752,16,40250
4107,16-41080,0.159360,3.133683,47.506667,0.103007,0.053081,0.003272,0.063929,15.402410,45.373778,16,41080
4489,17-44900,0.023849,4.554960,53.793261,0.017225,0.005775,0.000849,0.004996,17.831019,80.620578,17,44900


In [100]:
dims = [2,3,4,7]
np_x = np.random.uniform(size=dims).astype(np.float32)

In [31]:
90000/(32*32)

87.890625

In [30]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy'
np.load(fname,mmap_mode='r').shape


(101251, 1, 64, 64, 64)

In [32]:
! du -hs /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy



99G	/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy
